In [1]:
import itertools
import pandas as pd
import numpy as np
import scipy.stats
import re
import requests
import math
from bs4 import BeautifulSoup
from selenium import webdriver
import matplotlib as mlp
import matplotlib.pyplot as plt
import matplotlib.backends.backend_agg
import matplotlib.figure
#% matplotlib inline
import seaborn as sb
import datetime
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer
from sklearn.metrics import roc_curve, roc_auc_score

### HTML soups from all 500 web pages

In [2]:
def get_soups(website_number):
    get_url = requests.get('https://www.truecar.com/used-cars-for-sale/listings/?page=' + str(website_number))
    return BeautifulSoup(get_url.content, 'lxml')

In [3]:
soups = list(map(get_soups, list(range(2551, 2585))))

## URLs scraping

In [4]:
def urls_scraper(soup):
    nth_urls = []
    def urlsppage(nth):
        finding = soup.find_all('a', {'data-test': 'usedListing'})[nth]
        return re.findall('href="/.+" style', str(finding)[:280])[0]
    nth_urls = list(map(urlsppage, list(range(30))))
    return nth_urls

urls_list = list(map(urls_scraper, soups))
pages_urls = np.array(urls_list).flatten()
url_formatter = np.vectorize(lambda url: 'https://truecar.com' + url[6: -7])
urls = url_formatter(pages_urls)

In [5]:
def fscraper(url, feature):
    nth_request = requests.get(url)
    nth_soup = BeautifulSoup(nth_request.content, 'lxml')
    nth_search = re.search(feature + '</h4><ul><li>.+</li', str(nth_soup))
    try:
        return re.findall('li>.+</l', str(nth_search))[0][3: -3]
    except:
        return np.NaN

## First part of vehicles features scraping

In [6]:
drive_types = list(map(fscraper, urls, itertools.repeat('Drive Type')))
fuel_types = list(map(fscraper, urls, itertools.repeat('Fuel Type')))

In [7]:
mileages = list(map(fscraper, urls, itertools.repeat('Mileage')))
transmissions = list(map(fscraper, urls, itertools.repeat('Transmission')))

In [8]:
MPGs = list(map(fscraper, urls, itertools.repeat('MPG')))
options_levels = list(map(fscraper, urls, itertools.repeat('Options Level')))
bed_lengths = list(map(fscraper, urls, itertools.repeat('Bed Length')))

In [14]:
engines = list(map(fscraper, urls, itertools.repeat('Engine')))
exterior_colors = list(map(fscraper, urls, itertools.repeat('Exterior Color')))
interior_colors = list(map(fscraper, urls, itertools.repeat('Interior Color')))
styles = list(map(fscraper, urls, itertools.repeat('Style')))

ConnectionError: HTTPSConnectionPool(host='truecar.com', port=443): Max retries exceeded with url: /used-cars-for-sale/listing/5XXGT4L34KG285063/2019-kia-optima/ (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x14E49430>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

## Vehicles Years, Makes and Models scraping

In [11]:
def ymm_scraper(url, feat_name):
    nth_request = requests.get(url)
    nth_soup = BeautifulSoup(nth_request.content, 'lxml')
    nth_finding = nth_soup.find_all('div', {'class': 'text-truncate heading-3 margin-right-2 margin-right-sm-3'})
    try:
        if feat_name == 'year':
            return re.findall('>.+</', str(nth_finding))[0].split()[0][1:]
        elif feat_name == 'make':
            return ' '.join(re.findall('>.+</', str(nth_finding))[0].split()[1: -1])
        else:
            return re.findall('>.+</', str(nth_finding))[0].split()[-1][: -2]
    except:
        return np.NaN

In [13]:
years = list(map(ymm_scraper, urls, itertools.repeat('year')))
makes = list(map(ymm_scraper, urls, itertools.repeat('make')))
models = list(map(ymm_scraper, urls, itertools.repeat('model')))

ConnectionError: HTTPSConnectionPool(host='truecar.com', port=443): Max retries exceeded with url: /used-cars-for-sale/listing/5XXGT4L34KG285063/2019-kia-optima/ (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x14CFEB50>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

## Vehicles Prices scraping

In [ ]:
def prices_scraper(url):
    nth_request = requests.get(url).content
    nth_soup = BeautifulSoup(nth_request, 'lxml').find_all('div', {'data-qa': 'LabelBlock-text'})
    try:
        return re.findall('[0-9]+,[0-9]+', str(nth_soup))[0]
    except:
        return np.NaN

In [ ]:
prices = list(map(prices_scraper, urls))

## Vehicles Locations (Cities and States) scraping

In [ ]:
def cities_scraper(url):
    nth_request = requests.get(url).content
    nth_soup = BeautifulSoup(nth_request, 'lxml').find_all('span', {'data-qa': 'used-vdp-header-location'})
    try:
        return re.findall('">.+<!', str(nth_soup))[0][2: -12]
    except:
        return np.NaN

In [ ]:
def states_scraper(url):
    nth_request = requests.get(url).content
    nth_soup = BeautifulSoup(nth_request, 'lxml').find_all('span', {'data-qa': 'used-vdp-header-location'})
    try:
        return re.findall('[A-W][A-Z]', str(nth_soup))[0]
    except:
        return np.NaN

In [ ]:
cities = list(map(cities_scraper, urls))
states = list(map(states_scraper, urls))

## Vehicles Conditions scraping

In [ ]:
def conditions_scraper(url):
    nth_request = requests.get(url).content
    nth_soup = BeautifulSoup(nth_request, 'lxml').find_all('li', {'class': '_h9wfdq'})
    try:
        return re.findall('">[0-9]<!', str(nth_soup[0]))[0][2: -2] + re.findall('->.+</l', str(nth_soup[0]))[0][2: -3]
    except:
        return np.NaN

In [ ]:
conditions = list(map(conditions_scraper, urls))

## Dataset

In [ ]:
features = {
    'Make': makes, 'Model': models, 'Year': years, 'Mileage': mileages, 'Transmission': transmissions,
    'Engine': engines, 'Exterior Color': exterior_colors, 'Interior Color': interior_colors,
    'MPG': MPGs, 'Fuel Type': fuel_types, 'Drive Type': drive_types, 'Location (City)': cities,
    'Location (State)': states, 'Style': styles, 'Condition (Accidents)': conditions,
    'Options Level': options_levels, 'Bed Length': bed_lengths, 'Price': prices
}

In [ ]:
vehicles_data = pd.DataFrame(features)

In [ ]:
vehicles_data.to_csv('C:/Users/aleja/Desktop/v2551_2565.csv')